In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

# Load the dataset
df = pd.read_csv("/content/spam.csv", encoding='latin-1')
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
df.rename(columns={'v1': 'message_type', 'v2': 'message'}, inplace=True)

# Preprocessing and feature extraction
df['message_type'] = df['message_type'].map({'ham': 0, 'spam': 1})
df['num_characters'] = df['message'].apply(len)

# Text preprocessing functions
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text.lower())

    # Removing punctuation and stopwords
    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(stemmed_tokens)

# Apply preprocessing to messages
df['processed_message'] = df['message'].apply(preprocess_text)

# Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(df['processed_message']).toarray()
y = df['message_type']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Naive Bayes classifier training and evaluation
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)
y_pred = naive_bayes.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print("Naive Bayes Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)


Naive Bayes Accuracy: 0.9632286995515695
Confusion Matrix:
 [[965   0]
 [ 41 109]]
Precision: 1.0


In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

# Load the dataset
df = pd.read_csv("/content/spam.csv", encoding='latin-1')
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
df.rename(columns={'v1': 'message_type', 'v2': 'message'}, inplace=True)

# Preprocessing and feature extraction
df['message_type'] = df['message_type'].map({'ham': 0, 'spam': 1})
df['num_characters'] = df['message'].apply(len)

# Text preprocessing functions
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

# Apply preprocessing to messages
df['processed_message'] = df['message'].apply(preprocess_text)

# Define pipeline with TfidfVectorizer and MultinomialNB
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Define parameter grid for GridSearchCV
param_grid = {
    'tfidf__max_features': [100, 500, 1000, 2000, 3000, 5000],
    'tfidf__use_idf': [True, False],
    'tfidf__sublinear_tf': [True, False]
}

# Split the dataset
X = df['processed_message']
y = df['message_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print("Best Model Parameters:", grid_search.best_params_)
print("Best Model Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Model Parameters: {'tfidf__max_features': 1000, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': False}
Best Model Accuracy: 0.9650224215246637
Confusion Matrix:
 [[962   3]
 [ 36 114]]
Precision: 0.9743589743589743


Creating a website using Streamlit to showcase your spam detection model and its accuracy is a great idea! Streamlit is a Python library that allows you to create interactive web applications with minimal code. Below is a basic example of how you can create a Streamlit web app for your spam detection model:

First, make sure you have Streamlit installed. You can install it using the following command

In [ ]:
pip install streamlit


Create a Python file (e.g., app.py) with the following code:

In [ ]:
import streamlit as st
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load the pickled objects
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))
model = pickle.load(open('model.pkl', 'rb'))

# Function to preprocess input text
def preprocess_text(text):
    # Put your preprocessing code here (similar to what you used in training)
    return text

# Streamlit UI elements
st.title('Spam Detection Web App')
st.write('Enter a message to predict if it\'s spam or not:')

input_text = st.text_area('Enter your message', '')

if st.button('Predict'):
    if input_text:
        preprocessed_text = preprocess_text(input_text)
        transformed_text = vectorizer.transform([preprocessed_text])
        prediction = model.predict(transformed_text)[0]

        if prediction == 1:
            st.error('Spam')
        else:
            st.success('Not Spam')

# Additional section to showcase accuracy
st.subheader('Model Accuracy')
accuracy = 0.0  # Load and calculate accuracy here
st.write(f'Model Accuracy: {accuracy:.2%}')


Place the vectorizer.pkl and model.pkl files in the same directory as your app.py file.

Open a terminal, navigate to the directory containing app.py, and run the following command:

In [ ]:
streamlit run app.py


This will launch a web app in your browser where you can input text and get predictions from your model.
